In [1]:
!pip install kaggle



In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!ls ~/.kaggle


kaggle.json


In [5]:
!kaggle competitions download -c shl-intern-hiring-assessment-2025 -p /content


 99% 1.22G/1.23G [00:13<00:00, 343MB/s]
100% 1.23G/1.23G [00:13<00:00, 101MB/s]


In [6]:
!ls



kaggle.json  sample_data  shl-intern-hiring-assessment-2025.zip


In [7]:
!unzip shl-intern-hiring-assessment-2025.zip


Archive:  shl-intern-hiring-assessment-2025.zip
  inflating: dataset/audios/test/audio_1.wav  
  inflating: dataset/audios/test/audio_10.wav  
  inflating: dataset/audios/test/audio_100.wav  
  inflating: dataset/audios/test/audio_101.wav  
  inflating: dataset/audios/test/audio_102.wav  
  inflating: dataset/audios/test/audio_102_1.wav  
  inflating: dataset/audios/test/audio_103.wav  
  inflating: dataset/audios/test/audio_104.wav  
  inflating: dataset/audios/test/audio_105.wav  
  inflating: dataset/audios/test/audio_106.wav  
  inflating: dataset/audios/test/audio_106_1.wav  
  inflating: dataset/audios/test/audio_107.wav  
  inflating: dataset/audios/test/audio_108.wav  
  inflating: dataset/audios/test/audio_109.wav  
  inflating: dataset/audios/test/audio_10_1.wav  
  inflating: dataset/audios/test/audio_11.wav  
  inflating: dataset/audios/test/audio_110.wav  
  inflating: dataset/audios/test/audio_111.wav  
  inflating: dataset/audios/test/audio_112.wav  
  inflating: dataset

In [14]:
!ls dataset/audios
!ls dataset/csvs


test  train
test.csv  train.csv


In [15]:
import pandas as pd

train_df = pd.read_csv("dataset/csvs/train.csv")
test_df  = pd.read_csv("dataset/csvs/test.csv")

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

train_df.head()


Train shape: (409, 2)
Test shape: (197, 1)


,filename,label
0,audio_173,3.0
1,audio_138,3.0
2,audio_127,2.0
3,audio_95,2.0
4,audio_73,3.5


In [21]:
import librosa

y, sr = librosa.load(audio_path, sr=16000)

print("Sample rate:", sr)
print("Duration (seconds):", len(y)/sr)


/tmp/ipython-input-3728778292.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=16000)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Sample rate: 16000
Duration (seconds): 60.0746875


In [25]:
import librosa
import numpy as np

def extract_mfcc(file_path):
    y, sr = librosa.load(file_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    mfcc_mean = np.mean(mfcc.T, axis=0)
    return mfcc_mean


In [26]:
features = extract_mfcc(audio_path)
print("MFCC shape:", features.shape)


/tmp/ipython-input-2092584800.py:5: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=16000)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


MFCC shape: (40,)


In [28]:
from tqdm import tqdm
import numpy as np

X = []
y_labels = []

for _, row in tqdm(train_df.iterrows(), total=len(train_df)):
    file_path = f"dataset/audios/train/{row['filename']}.wav"
    mfcc_feat = extract_mfcc(file_path)
    X.append(mfcc_feat)
    y_labels.append(row["label"])

X = np.array(X)
y_labels = np.array(y_labels)

print("X shape:", X.shape)
print("y shape:", y_labels.shape)


  0%|          | 0/409 [00:00<?, ?it/s]/tmp/ipython-input-2092584800.py:5: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=16000)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
  0%|          | 1/409 [00:00<02:14,  3.03it/s]/tmp/ipython-input-2092584800.py:5: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=16000)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
  1%|          | 3/409 [00:00<01:31,  4.45it/s]/tmp/ipython-input-2092584800.py:5: UserWarning: Py

X shape: (409, 40)
y shape: (409,)


In [30]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y_labels,
    test_size=0.2,
    random_state=42
)

print(X_train.shape, X_val.shape)


(327, 40) (82, 40)


In [32]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    n_estimators=300,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)


RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=42)

In [33]:
from sklearn.metrics import mean_squared_error
import numpy as np

train_preds = model.predict(X_train)
train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))

print("Training RMSE:", train_rmse)


Training RMSE: 0.28183653021839694


In [34]:
val_preds = model.predict(X_val)
val_rmse = np.sqrt(mean_squared_error(y_val, val_preds))

print("Validation RMSE:", val_rmse)


Validation RMSE: 0.7358803237949154


In [37]:
X_test = []

for _, row in test_df.iterrows():
    file_path = f"dataset/audios/test/{row['filename']}.wav"
    mfcc_feat = extract_mfcc(file_path)
    X_test.append(mfcc_feat)

X_test = np.array(X_test)
print("Test feature shape:", X_test.shape)


/tmp/ipython-input-2092584800.py:5: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=16000)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipython-input-2092584800.py:5: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=16000)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipython-input-2092584800.py:5: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=16000)
/usr/local/lib/python3.12/dist-packages/libro

Test feature shape: (197, 40)


In [38]:
test_preds = model.predict(X_test)
print(test_preds[:10])


[3.455      2.66       2.93333333 2.85       2.795      3.13
 3.08333333 2.82666667 2.865      3.23666667]


In [41]:
from scipy.stats import pearsonr

pearson_corr, _ = pearsonr(y_val, val_preds)
print("Validation Pearson Correlation:", pearson_corr)


Validation Pearson Correlation: 0.30898029613357736


In [48]:
# Predict on test data
test_preds = model.predict(X_test)

# Create submission file
submission = pd.DataFrame({
    "filename": test_df["filename"],
    "label": test_preds
})

# Save with new filename
submission.to_csv("submission1.csv", index=False)

submission.head()

,filename,label
0,audio_141,3.455000
1,audio_114,2.660000
2,audio_17,2.933333
3,audio_76,2.850000
4,audio_156,2.795000


In [50]:
!rm kaggle.json
